In [2]:
#http://stackoverflow.com/questions/32503826/how-can-i-grab-data-series-from-xml-or-tcx-file
#https://github.com/vkurup/python-tcxparser/blob/master/tcxparser.py
#http://www.diveintopython3.net/xml.html
#TODO add wind bearing relative to rider
#TODO create class based structure for the data and functions!
#imports
import numpy as np
import tcxparser
import csv,json,requests
from pprint import pprint

In [14]:
class tcxWeather:
    def __init__(self, xmlfile):
        self.raw = tcxparser.TCXParser(xmlfile)
        self.stravaTime = self.raw.time_values()
        self.length = len(self.stravaTime)
        self.latitude = self.raw.latitude_points()
        self.longitude = self.raw.longitude_points()
        self.distance = self.raw.distance_points()
        
   
    def speed(self,**kwargs):
        mps_mph = 2.23694
        mps_kph = 3.6
        if 'mph' in kwargs:
            self.mph = kwargs['mph']
            self.mps = self.mph/mps_mph
            self.kph = self.mps*mps_kph
        if 'kph' in kwargs:
            self.kph = kwargs['kph']
            self.mps = self.kph/mps_kph
            self.mph = self.mps*mps_mph
        if 'mps' in kwargs:
            self.mps = kwargs['mps']
            self.kph = self.mps*mps_kph
            self.mph = self.mps*mps_mph
            
            
    def decimate(self,**kwargs): #todo add variable input parameters such as dist=10km or time = 1hr or numpoints = 20
        if 'Distance' in kwargs:
            distance = kwargs['Distance']
            distanceBetweenPoints = self.distance[1]-self.distance[0] #assumed constant should average or something
            numPoints = self.length*(distanceBetweenPoints/distance)
            numPoints = np.floor(numPoints).astype(int)
        if 'Points' in kwargs:
            numPoints = kwargs['Points']
        if 'Time' in kwargs:
            if hasattr(self, 'mps'):
                print('TODO')
            else:
                raise Exception('no speed defined use x.Speed(mps =y | kph =z | mph =w)')
            numPoints = 10
       
        print('Decimating to {0} Points'.format(numPoints))   
        ind = np.linspace(0, (self.length-1),numPoints, endpoint=True, retstep=False, dtype=None)
        ind = np.floor(ind)
        ind = ind.astype(int)
        self.lat = np.array(self.latitude)
        self.lon = np.array(self.longitude)
        self.dist = np.array(self.distance)
        self.lat = self.lat[np.ix_(ind)]
        self.lon = self.lon[np.ix_(ind)]
        self.dist = self.dist[np.ix_(ind)]
        

In [5]:
#Read the data
xmlfile = 'demoRoute.tcx'
data = tcxWeather

In [ ]:
lat = lat[4]
lon = lon[4]
units = 'si'

url = 'https://api.darksky.net/forecast/***REMOVED***/{0},{1}?units={2}'.format(lat,lon,units)

retrieved = requests.get(url)
file = open('weatherdataTest.json','wb') 
file.write(retrieved.content) 
file.close()

In [ ]:
with open('weatherdataTest.json') as data_file:    
    weatherdata = json.load(data_file)


In [20]:
steve = tcxWeather(xmlfile)
steve.decimate(Points=100)

Decimating to 100 Points


In [ ]:
#Bearing http://www.movable-type.co.uk/scripts/latlong.html
y = np.sin(λ2-λ1) * np.cos(φ2)
x = np.cos(φ1)*np.sin(φ2) - np.sin(φ1)*np.cos(φ2)*np.cos(λ2-λ1)
brng = Math.atan2(y, x).toDegrees();

In [ ]:
#pip install gpxpy or the maths!
#https://nssdc.gsfc.nasa.gov/planetary/factsheet/earthfact.html
#not actually needed as Strava has already supplied this data
def distance(lat1,lat2,lon1,lon2):
    "This calculates distance between two coordinates"
    R = 6371.008 #mean earth radius in km
    (latR1,latR2,lonR1,lonR2) = [np.deg2rad(x) for x in (lat1,lat2,lon1,lon2)]
    dlat = latR2-latR1
    dlon = lonR2-lonR1
    a = np.sin(dlat / 2)**2 + np.cos(latR1) * np.cos(latR2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    dist = R*c
    return dist;
